In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Loading the data
file_path = r"C:\Users\Lenovo\OneDrive - University of Leeds\Project\mimic-iv-clinical-database-demo-2.2\mimic-iv-clinical-database-demo-2.2\prescriptions.csv"
data = pd.read_csv(file_path)

print("First few rows of the dataset:")
print(data.head())

First few rows of the dataset:
   subject_id   hadm_id  pharmacy_id poe_id  poe_seq order_provider_id  \
0    10027602  28166872     27168639    NaN      NaN               NaN   
1    10027602  28166872     40720238    NaN      NaN               NaN   
2    10027602  28166872     62845687    NaN      NaN               NaN   
3    10027602  28166872     24340150    NaN      NaN               NaN   
4    10027602  28166872     14435820    NaN      NaN               NaN   

             starttime stoptime drug_type              drug  ...  gsn ndc  \
0  2201-10-30 12:00:00      NaN      MAIN  Fentanyl Citrate  ...  NaN NaN   
1  2201-10-30 12:00:00      NaN      MAIN  Fentanyl Citrate  ...  NaN NaN   
2  2201-10-31 12:00:00      NaN      MAIN         Lorazepam  ...  NaN NaN   
3  2201-10-30 12:00:00      NaN      MAIN         Midazolam  ...  NaN NaN   
4  2201-10-30 12:00:00      NaN      MAIN         Midazolam  ...  NaN NaN   

   prod_strength form_rx dose_val_rx dose_unit_rx form_val_di

In [2]:
null_values = data.isnull().sum()

total_columns = data.shape[1]
total_rows = data.shape[0]
print(f"Total columns: {total_columns}")
print(f"Total rows: {total_rows}")
print("-------------")

print(null_values)


Total columns: 21
Total rows: 18087
-------------
subject_id               0
hadm_id                  0
pharmacy_id              0
poe_id                 118
poe_seq                118
order_provider_id       94
starttime                0
stoptime                14
drug_type                0
drug                     0
formulary_drug_cd       12
gsn                   2519
ndc                     21
prod_strength            9
form_rx              18075
dose_val_rx              9
dose_unit_rx             9
form_val_disp            9
form_unit_disp           9
doses_per_24_hrs      7383
route                    6
dtype: int64


In [3]:
data.drop(columns=['form_rx'], inplace=True)


In [4]:

data['starttime'] = pd.to_datetime(data['starttime'], errors='coerce')
data['stoptime'] = pd.to_datetime(data['stoptime'], errors='coerce')



In [5]:
data['poe_id'].fillna('Unknown', inplace=True)
data['poe_seq'].fillna(0, inplace=True)
data['order_provider_id'].fillna('Unknown', inplace=True)
data['doses_per_24_hrs'].fillna(data['doses_per_24_hrs'].median(), inplace=True)  # Filling with median


In [7]:
data.info(), data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18087 entries, 0 to 18086
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   subject_id         18087 non-null  int64         
 1   hadm_id            18087 non-null  int64         
 2   pharmacy_id        18087 non-null  int64         
 3   poe_id             18087 non-null  object        
 4   poe_seq            18087 non-null  float64       
 5   order_provider_id  18087 non-null  object        
 6   starttime          18087 non-null  datetime64[ns]
 7   stoptime           18073 non-null  datetime64[ns]
 8   drug_type          18087 non-null  object        
 9   drug               18087 non-null  object        
 10  formulary_drug_cd  18075 non-null  object        
 11  gsn                15568 non-null  object        
 12  ndc                18066 non-null  float64       
 13  prod_strength      18078 non-null  object        
 14  dose_v

(None,
    subject_id   hadm_id  pharmacy_id   poe_id  poe_seq order_provider_id  \
 0    10027602  28166872     27168639  Unknown      0.0           Unknown   
 1    10027602  28166872     40720238  Unknown      0.0           Unknown   
 2    10027602  28166872     62845687  Unknown      0.0           Unknown   
 3    10027602  28166872     24340150  Unknown      0.0           Unknown   
 4    10027602  28166872     14435820  Unknown      0.0           Unknown   
 
             starttime stoptime drug_type              drug formulary_drug_cd  \
 0 2201-10-30 12:00:00      NaT      MAIN  Fentanyl Citrate            FENT2I   
 1 2201-10-30 12:00:00      NaT      MAIN  Fentanyl Citrate            FENT2I   
 2 2201-10-31 12:00:00      NaT      MAIN         Lorazepam            LORA2I   
 3 2201-10-30 12:00:00      NaT      MAIN         Midazolam            MIDA2I   
 4 2201-10-30 12:00:00      NaT      MAIN         Midazolam            MIDA2I   
 
    gsn  ndc prod_strength dose_val_rx do

In [8]:
null_values = data.isnull().sum()

total_columns = data.shape[1]
total_rows = data.shape[0]
print(f"Total columns: {total_columns}")
print(f"Total rows: {total_rows}")
print("-------------")

# Display the null values in each column
print(null_values)


Total columns: 20
Total rows: 18087
-------------
subject_id              0
hadm_id                 0
pharmacy_id             0
poe_id                  0
poe_seq                 0
order_provider_id       0
starttime               0
stoptime               14
drug_type               0
drug                    0
formulary_drug_cd      12
gsn                  2519
ndc                    21
prod_strength           9
dose_val_rx             9
dose_unit_rx            9
form_val_disp           9
form_unit_disp          9
doses_per_24_hrs        0
route                   6
dtype: int64


In [9]:
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer
import time

In [10]:
df_data = data.head(2000)


In [11]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df_data)


In [12]:
custom_synthesizer = CTGANSynthesizer(metadata, epochs=1000)

start_time = time.time()

# Training the synthesizer on the real data
custom_synthesizer.fit(df_data)

end_time = time.time()

elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")



Training time: 1654.0911722183228 seconds


In [13]:
# Generating synthetic data
synthetic_data = custom_synthesizer.sample(1000)  # Generate 1000 rows of synthetic data

# Displaying the first few rows of synthetic data
print(synthetic_data.head())

   subject_id   hadm_id  pharmacy_id         poe_id  poe_seq  \
0    10015338  25853702     49857517  sdv-pii-fnwyq    577.0   
1    10020568  27198653     25281963  sdv-pii-38eyu    116.0   
2    10037360  27838579     13809890  sdv-pii-l5fxh    202.0   
3    10026705  23180943     23076466  sdv-pii-y3pi7    462.0   
4    10021028  29732658     27750829  sdv-pii-dtqnh    108.0   

  order_provider_id           starttime            stoptime drug_type  \
0     sdv-pii-0xi01 2149-11-14 14:37:52 2120-12-02 18:10:13      MAIN   
1     sdv-pii-3egq9 2152-01-09 16:47:23 2187-06-07 03:07:18      MAIN   
2     sdv-pii-2rih0 2154-09-15 09:00:32 2148-08-17 00:35:34      MAIN   
3     sdv-pii-okm0a 2146-11-05 05:47:59 2150-11-29 20:08:47      MAIN   
4     sdv-pii-87skc 2188-04-30 20:50:42 2151-02-10 18:05:24      MAIN   

                                           drug formulary_drug_cd  \
0                            Losartan Potassium            CLOP75   
1                  Clonidine Patch 0.2

In [14]:
synthetic_data['subject_id'] = range(400000, 400000 + len(synthetic_data))
synthetic_data['hadm_id'] = synthetic_data['subject_id'].apply(lambda x: f"{x}{str(synthetic_data.index.get_loc(synthetic_data.index[synthetic_data['subject_id'] == x][0])).zfill(2)}")



In [26]:
synthetic_data.to_csv('synthetic_prescription.csv', index=False)

print("Synthetic data generated, updated, and saved successfully.")

Synthetic data generated, updated, and saved successfully.


In [16]:
null_values = synthetic_data.isnull().sum()

total_columns = synthetic_data.shape[1]
total_rows = synthetic_data.shape[0]
print(f"Total columns: {total_columns}")
print(f"Total rows: {total_rows}")
print("-------------")

print(null_values)


Total columns: 20
Total rows: 1000
-------------
subject_id             0
hadm_id                0
pharmacy_id            0
poe_id                 0
poe_seq                0
order_provider_id      0
starttime              0
stoptime               6
drug_type              0
drug                   0
formulary_drug_cd      5
gsn                  144
ndc                    8
prod_strength         36
dose_val_rx           34
dose_unit_rx          30
form_val_disp         36
form_unit_disp        35
doses_per_24_hrs       0
route                 21
dtype: int64


In [20]:
from sdv.evaluation.single_table import run_diagnostic

diagnostic = run_diagnostic(
    real_data=data,
    synthetic_data=synthetic_data,
    metadata=metadata
)


Generating report ...

(1/2) Evaluating Data Validity: |████████████████████████████████████████████████████| 20/20 [00:00<00:00, 404.72it/s]|
Data Validity Score: 94.12%

(2/2) Evaluating Data Structure: |█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 322.64it/s]|
Data Structure Score: 100.0%

Overall Score (Average): 97.06%



In [21]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    data,
    synthetic_data,
    metadata
)

Generating report ...

(1/2) Evaluating Column Shapes: |████████████████████████████████████████████████████| 20/20 [00:00<00:00, 136.44it/s]|
Column Shapes Score: 57.83%

(2/2) Evaluating Column Pair Trends: |██████████████████████████████████████████████| 190/190 [00:33<00:00,  5.62it/s]|
Column Pair Trends Score: 43.63%

Overall Score (Average): 50.73%



In [22]:
quality_report.get_details('Column Shapes')

,Column,Metric,Score,Error
0,subject_id,KSComplement,0.000000,None
1,hadm_id,KSComplement,NaN,TypeError: '<' not supported between instances...
2,pharmacy_id,KSComplement,0.805124,None
3,poe_seq,KSComplement,0.937979,None
4,starttime,KSComplement,0.855818,None
5,stoptime,KSComplement,0.845656,None
6,drug_type,TVComplement,0.926654,None
7,drug,TVComplement,0.245343,None
8,formulary_drug_cd,TVComplement,0.175859,None
9,gsn,TVComplement,0.169410,None


In [24]:
from sdv.evaluation.single_table import get_column_plot

fig = get_column_plot(
    real_data=data,
    synthetic_data=synthetic_data,
    column_name='drug',
    metadata=metadata
)

fig.show()

In [25]:
import pandas as pd

# Loading the new provided data
data = [
    {'Column': 'subject_id', 'Metric': 'KSComplement', 'Score': 0.0, 'Error': None},
    {'Column': 'hadm_id', 'Metric': 'KSComplement', 'Score': None, 'Error': "TypeError: '<' not supported between instances"},
    {'Column': 'pharmacy_id', 'Metric': 'KSComplement', 'Score': 0.805124, 'Error': None},
    {'Column': 'poe_seq', 'Metric': 'KSComplement', 'Score': 0.937979, 'Error': None},
    {'Column': 'starttime', 'Metric': 'KSComplement', 'Score': 0.855818, 'Error': None},
    {'Column': 'stoptime', 'Metric': 'KSComplement', 'Score': 0.845656, 'Error': None},
    {'Column': 'drug_type', 'Metric': 'TVComplement', 'Score': 0.926654, 'Error': None},
    {'Column': 'drug', 'Metric': 'TVComplement', 'Score': 0.245343, 'Error': None},
    {'Column': 'formulary_drug_cd', 'Metric': 'TVComplement', 'Score': 0.175859, 'Error': None},
    {'Column': 'gsn', 'Metric': 'TVComplement', 'Score': 0.169410, 'Error': None},
    {'Column': 'ndc', 'Metric': 'KSComplement', 'Score': 0.681090, 'Error': None},
    {'Column': 'prod_strength', 'Metric': 'TVComplement', 'Score': 0.250446, 'Error': None},
    {'Column': 'dose_val_rx', 'Metric': 'TVComplement', 'Score': 0.447837, 'Error': None},
    {'Column': 'dose_unit_rx', 'Metric': 'TVComplement', 'Score': 0.704732, 'Error': None},
    {'Column': 'form_val_disp', 'Metric': 'TVComplement', 'Score': 0.520522, 'Error': None},
    {'Column': 'form_unit_disp', 'Metric': 'TVComplement', 'Score': 0.713748, 'Error': None},
    {'Column': 'doses_per_24_hrs', 'Metric': 'TVComplement', 'Score': 0.925400, 'Error': None},
    {'Column': 'route', 'Metric': 'TVComplement', 'Score': 0.626065, 'Error': None}
]

df = pd.DataFrame(data)

thresholds = {
    'KSComplement': 0.7,
    'TVComplement': 0.7,
    'critical_columns': {
        'admittime': 0.8,
        'dischtime': 0.8,
        'admission_type': 0.9,
        'admission_location': 0.9,
        'discharge_location': 0.9,
        'insurance': 0.9,
        'marital_status': 0.9,
    },
    'non_critical_columns': {
        'subject_id': 0.0,
        'language': 0.0,
        'race': 0.0,
        'edregtime': 0.7,
        'edouttime': 0.7,
    }
}

def check_thresholds(df, thresholds):
    results = []
    for _, row in df.iterrows():
        column_name = row['Column']
        metric = row['Metric']
        score = row['Score']

        if column_name in thresholds['critical_columns']:
            threshold = thresholds['critical_columns'][column_name]
        else:
            threshold = thresholds['non_critical_columns'].get(column_name, thresholds.get(metric, 0))

        if pd.isna(score):
            results.append(f"{column_name} does not have a valid score. Error: {row['Error']}")
        elif score >= threshold:
            results.append(f"{column_name} meets the threshold with a score of {score:.2e}.")
        else:
            results.append(f"{column_name} does not meet the threshold. Score: {score:.2e}, Threshold: {threshold:.2e}")
    
    return results

results = check_thresholds(df, thresholds)

for result in results:
    print(result)


subject_id meets the threshold with a score of 0.00e+00.
hadm_id does not have a valid score. Error: TypeError: '<' not supported between instances
pharmacy_id meets the threshold with a score of 8.05e-01.
poe_seq meets the threshold with a score of 9.38e-01.
starttime meets the threshold with a score of 8.56e-01.
stoptime meets the threshold with a score of 8.46e-01.
drug_type meets the threshold with a score of 9.27e-01.
drug does not meet the threshold. Score: 2.45e-01, Threshold: 7.00e-01
formulary_drug_cd does not meet the threshold. Score: 1.76e-01, Threshold: 7.00e-01
gsn does not meet the threshold. Score: 1.69e-01, Threshold: 7.00e-01
ndc does not meet the threshold. Score: 6.81e-01, Threshold: 7.00e-01
prod_strength does not meet the threshold. Score: 2.50e-01, Threshold: 7.00e-01
dose_val_rx does not meet the threshold. Score: 4.48e-01, Threshold: 7.00e-01
dose_unit_rx meets the threshold with a score of 7.05e-01.
form_val_disp does not meet the threshold. Score: 5.21e-01, T